In [0]:
pip install azure-storage-file-datalake

In [0]:
# Import modules and libraries
import pandas as pd
from azure.storage.filedatalake import DataLakeServiceClient
from datetime import datetime

In [0]:
# Azure Storage connection details (using SAS token)
account_name = "cdmo"
sas_token = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-01-30T12:53:06Z&st=2025-01-30T04:53:06Z&spr=https&sig=AxwIrOD2cUZkmCPUEaby1ZPw%2Bu9DE7nksIUknqfytD8%3D" # Replace with your SAS token
container_name = "config"

# Data for the configuration file with the additional "UniqueKey" column
data = [
    ["bronze", "00-landing", "data/incoming/customerfeedback/", "csv", "01-bronze", "customerfeedback/", "delta", "FeedbackID", "true"],
    ["bronze", "00-landing", "data/incoming/manufacturebatch/", "csv", "01-bronze", "manufacturebatch/", "delta", "BatchID", "true"],
    ["bronze", "00-landing", "data/incoming/productformula/", "csv", "01-bronze", "productformula/", "delta", "ProductID", "true"],
    ["bronze", "00-landing", "data/incoming/sales/", "csv", "01-bronze", "sales/", "delta", "OrderID", "true"],
    ["bronze", "00-landing", "data/incoming/supplier/", "csv", "01-bronze", "supplier/", "delta", "SupplierID", "true"],
    ["silver", "01-bronze", "customerfeedback/", "delta", "02-silver", "customerfeedback_transformed/", "delta", "FeedbackID", "true"],
    ["silver", "01-bronze", "manufacturebatch/", "delta", "02-silver", "manufacturebatch_transformed/", "delta", "BatchID", "true"],
    ["silver", "01-bronze", "productformula/", "delta", "02-silver", "productformula_transformed/", "delta", "ProductID", "true"],
    ["silver", "01-bronze", "sales/", "delta", "02-silver/sales/", "delta", "OrderID", "true"],
    ["silver", "01-bronze", "supplier/", "delta", "02-silver/supplier/", "delta", "SupplierID", "true"],
    ["gold", "02-silver", "customerfeedback/", "delta", "03-gold", "customer_feedback_aggregated/", "delta", "FeedbackID", "true"],
    ["gold", "02-silver", "manufacturebatch/", "delta", "03-gold", "manufacture_batch_enriched/", "delta", "BatchID", "true"],
    ["gold", "02-silver", "productformula/", "delta", "03-gold", "product_formula_enriched/", "delta", "ProductID", "true"],
    ["gold", "02-silver", "sales/", "delta", "03-gold", "sales_aggregated/", "delta", "OrderID", "true"],
    ["gold", "02-silver", "supplier/", "delta", "03-gold", "supplier_data_enriched/", "delta", "SupplierID", "true"],
]

# Column names for the configuration file
columns = [
    "Layer", "SourceContainer", "SourcePath", "SourceFormat",
    "TargetContainer", "TargetPath", "TargetFormat", "UniqueKey", "AddTimestamp"
]

# Create a DataFrame from the data
df = pd.DataFrame(data, columns=columns)

# Append the current date to the metadata config file name
current_date = datetime.now().strftime("%Y%m%d")
config_file_name = f"metadata_config_{current_date}.csv"

# Save the DataFrame to a local CSV file
df.to_csv(config_file_name, index=False)
print(f"✅ Configuration file '{config_file_name}' created successfully.")

# Function to upload a file to Azure Data Lake Storage (ADLS)
def upload_to_adls(local_file_name, container_name):
    """
    Uploads a local file to Azure Data Lake Storage (ADLS) using SAS token authentication.

    Args:
        local_file_name (str): The local file name to upload.
        container_name (str): The name of the target ADLS container.
    """
    try:
        # Initialize the DataLakeServiceClient with the SAS token
        service_client = DataLakeServiceClient(
            account_url=f"https://{account_name}.dfs.core.windows.net",
            credential=sas_token,
        )

        # Get the file system client for the container
        file_system_client = service_client.get_file_system_client(container_name)

        # Upload the file directly into the container without creating a subdirectory
        with open(local_file_name, "rb") as data:
            file_client = file_system_client.get_file_client(local_file_name)
            file_client.upload_data(data, overwrite=True)

        print(f"✅ File '{local_file_name}' uploaded to ADLS in container '{container_name}'.")
    except Exception as e:
        print(f"❌ Error uploading file to ADLS: {e}")

# Call the function to upload the configuration file to ADLS
upload_to_adls(config_file_name, container_name)


✅ Configuration file 'metadata_config_20250130.csv' created successfully.
✅ File 'metadata_config_20250130.csv' uploaded to ADLS in container 'config'.
